In [ ]:
!pip -q install -U transformers peft accelerate safetensors huggingface_hub

In [ ]:
from huggingface_hub import login
login()  # paste your HF token when prompted

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_id = "mistralai/Mistral-Small-24B-Instruct-2501"     # base model you trained on
adapter_dir = "/content/mistral24b-qlora-adapter-final"   # upload/unzip your adapter here
merged_out = "/content/mistral24b-merged-bf16"            # output folder

# Tokenizer (use the regex fix)
tokenizer = AutoTokenizer.from_pretrained(
    base_id,
    use_fast=True,
    fix_mistral_regex=True,
)

# Load base in bf16 for merging (NOT 4-bit here)
base_model = AutoModelForCausalLM.from_pretrained(
    base_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Load adapter and merge
peft_model = PeftModel.from_pretrained(base_model, adapter_dir)
merged_model = peft_model.merge_and_unload()

# Save merged model + tokenizer
merged_model.save_pretrained(merged_out, safe_serialization=True)
tokenizer.save_pretrained(merged_out)

print("Merged model saved to:", merged_out)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

...

model-00009-of-00010.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Merged model saved to: /content/mistral24b-merged-bf16


In [ ]:
repo_id = "br2835/mistraladaptmerged"  # <- change this
merged_model.push_to_hub(repo_id, private=True)
tokenizer.push_to_hub(repo_id, private=True)

print("Pushed merged model to:", repo_id)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00010.safetensors:   0%|          |  613kB / 4.78GB            

  ...0009-of-00010.safetensors:   0%|          |  613kB / 4.78GB            

...

  ...0008-of-00010.safetensors:   0%|          | 4.29MB / 4.78GB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mptgc5b3z7/tokenizer.json: 100%|#########9| 17.0MB / 17.1MB            

Pushed merged model to: br2835/mistraladaptmerged


In [ ]:
# 1) Install deps (CUDA backend for MLX + mlx-lm)
!pip -q install -U "mlx[cuda12]" mlx-lm huggingface_hub

# 2) Login to Hugging Face (needed to read your PRIVATE merged repo + upload output)
# from huggingface_hub import login
# login()  # paste your HF token

# # (Optional) Put HF cache somewhere explicit on local disk
# import os
# os.environ["HF_HOME"] = "/content/hf"
# os.environ["HF_HUB_CACHE"] = "/content/hf/hub"
# os.environ["TRANSFORMERS_CACHE"] = "/content/hf/transformers"

# 3) Convert + 3-bit quantize + upload to a new repo
MERGED_REPO = "br2835/mistraladaptmerged"          # your merged bf16 repo (private ok)
UPLOAD_REPO = "br2835/mistraladaptmerged-mlx-3bit" # new repo to create (private by default)

!mlx_lm.convert \
  --hf-path {MERGED_REPO} \
  --mlx-path /content/mlx_out \
  -q --q-bits 3 \
  --upload-repo {UPLOAD_REPO}

Streaming output truncated to the last 5000 lines.
model-00009-of-00010.safetensors:  76% 3.62G/4.78G [00:34<00:04, 265MB/s]




model-00010-of-00010.safetensors:  90% 3.50G/3.90G [00:33<00:01, 298MB/s]
model-00009-of-00010.safetensors:  78% 3.71G/4.78G [00:34<00:03, 304MB/s]




model-00010-of-00010.safetensors:  91% 3.57G/3.90G [00:34<00:01, 283MB/s]
model-00009-of-00010.safetensors:  79% 3.78G/4.78G [00:36<00:08, 118MB/s]
model-00009-of-00010.safetensors:  80% 3.85G/4.78G [00:36<00:06, 151MB/s]




model-00010-of-00010.safetensors:  93% 3.63G/3.90G [00:36<00:02, 110MB/s]
model-00009-of-00010.safetensors:  82% 3.91G/4.78G [00:37<00:07, 116MB/s]
model-00009-of-00010.safetensors:  83% 3.98G/4.78G [00:37<00:05, 137MB/s]




model-00010-of-00010.safetensors:  95% 3.70G/3.90G [00:37<00:02, 90.9MB/s]




model-00010-of-00010.safetensors:  98% 3.83G/3.90G [00:37<00:00, 147MB/s] 
model-00009-of-00010.safetensors:  85% 4.05G/4.78G [00:37<00:04, 153MB/s]
model-00009-of-00010.safetensors:  86% 